### Tanmay Bhatt
### 011499072
CMPE 258 <br />
Assignmnt - 5 <br />
Date: 04/08/2018 <br />

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import h5py

## 1. (70pts) Define functions 
Please define the functions which are needed for CNN architectures. The following list is a suggestion.
One-hot encoding<br/>
Activation forward (Relu, sigmoid)<br/>
Compute cost<br/>
Zero pad<br/>
Convolution with single step<br/>
Convolution forward (for all data)<br/>
Pooling forward (max, average)<br/>
Pooling backward (max, average)<br/>
Activation backward (Relu, sigmoid)<br/>
Convolution backward (for all data)<br/>
Forward propagation (including all steps)<br/>
Backward propagation (including all steps)<br/>
Parameter updating (Gradient descent or other optimization method)<br/>

In [2]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [3]:
def sigmoid_derivative(a):
    return a * (1-a)

In [4]:
def ReLU_Derivative(a):
    a[a<=0] = 0
    return a

In [5]:
def ReLU(z):
    return z * (z > 0)

In [6]:
def zero_pad(X,p):
    return np.pad(X, [(0,0), (p,p), (p,p), (0,0)], 'constant', constant_values=(0))

In [7]:
def flatten_array(X):
    temp = []
    for item in X:
        temp.append(item.flatten())
    return np.asarray(temp)

In [8]:
def one_hot_encoding(mat):
    list_of_list = []
    for i in range(0,len(mat)):
        small_list = np.zeros(np.max(mat)+1)
        small_list[mat[i]] = 1
        list_of_list.append(small_list)
    result = np.array(list_of_list)
    return result

In [9]:
def conv_single_step(a_slice_prev, W, b):
   
    s = np.multiply(a_slice_prev, W) + b
    Z = np.sum(s)

    return Z

In [10]:
def conv_forward(A_prev, W, b, hparameters,activation="sigmoid"):
   
    
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    (f, f, n_C_prev, n_C) = W.shape

    stride = hparameters['stride']
    pad = hparameters['padding']
    
    n_H = int((n_H_prev - f + 2 * pad) / stride) + 1
    n_W = int((n_W_prev - f + 2 * pad) / stride) + 1
    
    Z = np.zeros((m, n_H, n_W, n_C))
    
    A_prev_pad = zero_pad(A_prev, pad)
    
    for i in range(m):                                 
        a_prev_pad = A_prev_pad[i]                     
        for h in range(n_H):                           
            for w in range(n_W):                       
                for c in range(n_C):                   

                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f
                    a_slice_prev = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]
                    Z[i, h, w, c] = conv_single_step(a_slice_prev, W[...,c], b[...,c])
                                            
    cache = (A_prev, W, b, hparameters)
    
    if activation == "sigmoid":
        a = sigmoid(Z)
    elif activation == "ReLU":
        a = ReLU(Z)

    return a, cache

In [11]:
def conv_backward(dZ, cache):
  
   
    (A_prev, W, b, hparameters) = cache
    
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    (f, f, n_C_prev, n_C) = W.shape
    
    stride = hparameters["stride"]
    pad = hparameters["padding"]
    
    (m, n_H, n_W, n_C) = dZ.shape
    
    dA_prev = np.zeros((m, n_H_prev, n_W_prev, n_C_prev))                           
    dW = np.zeros((f, f, n_C_prev, n_C))
    db = np.zeros((1, 1, 1, n_C))

    A_prev_pad = zero_pad(A_prev, pad)
    dA_prev_pad = zero_pad(dA_prev, pad)
    
    for i in range(m):                      
        
        a_prev_pad = A_prev_pad[i]
        da_prev_pad = dA_prev_pad[i]
        
        for h in range(n_H):                   
            for w in range(n_W):               
                for c in range(n_C):           
                    
                    vert_start = h
                    vert_end = vert_start + f
                    horiz_start = w
                    horiz_end = horiz_start + f
                    
                    a_slice = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]

                    da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, h, w, c]
                    dW[:,:,:,c] += a_slice * dZ[i, h, w, c]
                    db[:,:,:,c] += dZ[i, h, w, c]
    
    return dA_prev, dW, db

In [12]:
def pool_forward(A_prev, hparameters, mode = "max"):
    
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    f = hparameters["f"]
    stride = hparameters["stride"]
    
    n_H = int(1 + (n_H_prev - f) / stride)
    n_W = int(1 + (n_W_prev - f) / stride)
    n_C = n_C_prev
    
    A = np.zeros((m, n_H, n_W, n_C))              
    
    for i in range(m):                           
        for h in range(n_H):                     
            for w in range(n_W):                
                for c in range (n_C):           
                    
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f
                    
                    a_prev_slice = A_prev[i, vert_start:vert_end, horiz_start:horiz_end, c]
                    
                    if mode == "max":
                        A[i, h, w, c] = np.max(a_prev_slice)
                    elif mode == "average":
                        A[i, h, w, c] = np.mean(a_prev_slice)
    
    
    cache = (A_prev, hparameters)
        
    return A, cache

In [13]:
def pool_backward(dA, cache, mode = "max"):
    
    
    (A_prev, hparameters) = cache
    
    stride = hparameters["stride"]
    f = hparameters["f"]
    
    m, n_H_prev, n_W_prev, n_C_prev = A_prev.shape
    m, n_H, n_W, n_C = dA.shape
    
    dA_prev = np.zeros(A_prev.shape)
    
    for i in range(m):                       
        a_prev = A_prev[i]
        for h in range(n_H):                   
            for w in range(n_W):               
                for c in range(n_C):          
                    vert_start = h
                    vert_end = vert_start + f
                    horiz_start = w
                    horiz_end = horiz_start + f
                    
                    if mode == "max":
                        a_prev_slice = a_prev[vert_start:vert_end, horiz_start:horiz_end, c]
                        mask = a_prev_slice == np.max(a_prev_slice)
                        dA_prev[i, vert_start:vert_end, horiz_start:horiz_end, c] += np.multiply(mask, dA[i, h, w, c])
                        
                    elif mode == "average":
                        da = dA[i, h, w, c]
                        shape = (f, f)
                        (n_H, n_W) = shape
                        dA_prev[i, vert_start:vert_end, horiz_start:horiz_end, c] += np.ones(shape) * da / (n_H * n_W)
                            
    return dA_prev

In [14]:
def forward_pass(X_mat):
    
    global neural_dict
    
    W1 = neural_dict['Fc1']['W']
    W2 = neural_dict['Fc2']['W']
    B1 = neural_dict['Fc1']['B']
    B2 = neural_dict['Fc2']['B']
    
    z1 = np.dot(X_mat,W1.T).T + B1
    a1 = ReLU(z1)
    z2 = (np.dot(W2, a1)+ B2)
    a2 = sigmoid(z2).T
    neural_dict['Fc1']['a'] = a1
    neural_dict['Fc2']['a'] = a2
    
    return a2

In [15]:
def backward_pass(f1):
    m = X_train.shape[0]
    y = Y_train_onehot
    global neural_dict

    W1 = neural_dict['Fc1']['W']
    W2 = neural_dict['Fc2']['W']
    B1 = neural_dict['Fc1']['B']
    B2 = neural_dict['Fc2']['B']
    a1 = neural_dict['Fc1']['a']
    a2 = neural_dict['Fc2']['a']

    dl_dz2 = a2-y
    dl_dw2 = np.dot(dl_dz2.T, a1.T)/m
    dl_db2 = np.sum(dl_dz2.T, axis=1, keepdims=True)/m
    dl_da1 = np.dot(dl_dz2, W2)
    dl_dz1 = np.multiply(dl_da1.T,ReLU_Derivative(a1))
    dl_dw1 = np.dot(dl_dz1, f1)/m
    dl_db1 = np.sum(dl_dz1, axis=1, keepdims=True)/m
    
    return dl_dw1, dl_dw2, dl_db1, dl_db2, dl_dz1

In [16]:
def calculate_loss():
    global neural_dict
    a = neural_dict['Fc2']['a']
    y = Y_train_onehot
    return (np.multiply(y,np.log(a)) + np.multiply((1-y),np.log(1-a)))

In [17]:
def calculate_cost():
    m = X_train.shape[0]
    cost = -np.sum(calculate_loss())
    return cost/m

In [18]:
def gradient_descent(X_train,learning_rate,iterations=1):
   
    global plot_object
    global neural_dict
    
    all_costs = []
    count = 0
    a1,c1 = conv_forward(X_train, neural_dict['Conv1']['W'], neural_dict['Conv1']['B'], neural_dict['Conv1']['params'],"ReLU")

    P1,cp1 = pool_forward(a1, neural_dict['Pool1']['params'], mode = "max")
    
    a2,c2 = conv_forward(P1, neural_dict['Conv2']['W'], neural_dict['Conv2']['B'], neural_dict['Conv2']['params'],"ReLU")
   
    P2,cp2 = pool_forward(a2,  neural_dict['Pool2']['params'], mode = "average")
    
    f1 = flatten_array(P2)

    forward_pass(f1)
    
    new_cost = calculate_cost()
    current_cost = float("inf")
    while new_cost <= current_cost and count < iterations:
        print "Iteration : ", count
        all_costs.append(new_cost)
        
        dl_dw1, dl_dw2, dl_db1, dl_db2, dl_dz1 = backward_pass(f1)
        
        df1 = np.dot(dl_dz1.T,neural_dict['Fc1']['W'])
        
        
        neural_dict['Fc1']['W'] = neural_dict['Fc1']['W'] - (learning_rate * dl_dw1)
        neural_dict['Fc2']['W'] = neural_dict['Fc2']['W'] - (learning_rate * dl_dw2)
        neural_dict['Fc1']['B'] = neural_dict['Fc1']['B'] - (learning_rate * dl_db1)
        neural_dict['Fc2']['B'] = neural_dict['Fc2']['B'] - (learning_rate * dl_db2)
       

        dA2 = df1.reshape(P2.shape)
        dA2 = pool_backward(dA2, cp2, mode = "average")
        dz2 = np.multiply(dA2,ReLU_Derivative(a2))
        dA1, dW2, dB2 = conv_backward(dz2, c2)
        dA1 = pool_backward(dA1, cp1, mode = "max")
        dz1 = np.multiply(dA1,ReLU_Derivative(a1))
        dA0, dW1, dB1 = conv_backward(dz1, c1)
        

        neural_dict['Conv1']['W'] = neural_dict['Conv1']['W'] - (learning_rate * dW1)
        neural_dict['Conv2']['W'] = neural_dict['Conv2']['W'] - (learning_rate * dW2)
        neural_dict['Conv1']['B'] = neural_dict['Conv1']['B'] - (learning_rate * dB1)
        neural_dict['Conv2']['B'] = neural_dict['Conv2']['B'] - (learning_rate * dB2)
        
        current_cost = new_cost
        
        a1,c1 = conv_forward(X_train, neural_dict['Conv1']['W'], neural_dict['Conv1']['B'], neural_dict['Conv1']['params'],"ReLU")
     
        P1,cp1 = pool_forward(a1, neural_dict['Pool1']['params'], mode = "max")
   
        a2,c2 = conv_forward(P1, neural_dict['Conv2']['W'], neural_dict['Conv2']['B'], neural_dict['Conv2']['params'],"ReLU")
   
        P2,cp2 = pool_forward(a2,  neural_dict['Pool2']['params'], mode = "average")
   
        f1 = flatten_array(P2)

        forward_pass(f1)
        
        new_cost = calculate_cost()
        print new_cost
        count +=1
    plot_object[learning_rate] = all_costs
    print "Final cost : ",
    print new_cost
    print "Iternations : %d" % count

## 2. Load data
Using Jupyter notebook, load the data.

In [19]:
X_train = np.load("./ex5_train_x.npy")
Y_train = np.load("./ex5_train_y.npy")
print 'train x shape :', X_train.shape
print 'train y shape :', Y_train.shape

train x shape : (1020, 64, 64, 3)
train y shape : (1020,)


## 3. (10pts) Initialize parameters (Weights, bias for each layer)
Please initialize weight coefficients and bias terms for each layer.
Please make sure the size (dimension) of each Weights and bias.
Please consider optimum initialization method depending on Activation function.
You may use your trained weights and bias. In this case, please make sure to submit the trained weights and bias
as one separate file (para_yourFirstName_LastnName)

In [20]:
Y_train_onehot = one_hot_encoding(Y_train)

In [21]:
X_train = X_train/255.0

In [22]:
hidden_neurons = 108
output_neurons = 6
np.random.seed(1)

W1_fc = []
for i in range(0,hidden_neurons):
    sampl = np.random.uniform(low=0, high=1, size=(1296)) * 0.01
    W1_fc.append(sampl)
W2_fc = []
for i in range(0,output_neurons):
    sampl = np.random.uniform(low=0, high=1, size=(hidden_neurons)) * 0.01
    W2_fc.append(sampl)
B1_fc = []
for i in range(0,hidden_neurons):
    B1_fc.append([0])
B2_fc = []
for i in range(0,output_neurons):
    B2_fc.append([0])
    
W1_fc = np.array(W1_fc)
W2_fc = np.array(W2_fc)
B1_fc = np.array(B1_fc)
B2_fc = np.array(B2_fc)


In [23]:
np.random.seed(1)
neural_dict = {
    
    'Conv1':{
        'W' : np.random.rand(4,4,3,8) * 0.01,
        'B' : np.random.rand(1,1,1,8) * 0.01,
        'params' : {
            'padding' : 1,
            'stride' : 2
        }
    },
    'Pool1' : {
        'params' : {
            'f' : 5,
            'stride' : 1
        }
    },
    'Conv2' : {
        
        'W' : np.random.rand(4,4,8,16) * 0.01,
        'B' : np.random.rand(1,1,1,16) * 0.01,
        'params' : {
            'padding' : 0,
            'stride' : 2
        }
    },
    'Pool2' : {
        'params' : {
            'f' : 5,
            'stride' : 1
        }
    },
    'Fc1' : {
        'W' : W1_fc,
        'B' : B1_fc
    },
    'Fc2' : {
        'W' : W2_fc,
        'B' : B2_fc
    }
}

In [24]:
iterations = 7
learning_rate = 1
plot_object = {}
start = datetime.now()
## Learning rate 1
gradient_descent(X_train,learning_rate,iterations=iterations)
end = datetime.now()
plot_object
print "Time taken : ", (end-start)

Iteration :  0
3.04980915232
Iteration :  1
2.93061792507
Iteration :  2
2.85571357037
Iteration :  3
2.80729839931
Iteration :  4
2.7752704456
Iteration :  5
2.75367706657
Iteration :  6
2.73889074499
Final cost :  2.73889074499
Iternations : 7
Time taken :  1:02:40.312724


In [25]:
print "Learning rate : ", iterations
print "Iterations ", learning_rate
print "Final Cost :", calculate_cost()

Learning rate :  7
Iterations  1
Final Cost : 2.73889074499


## 4. (20pts) Optimization of Convolution Neural Network model
Please build your model with forward propagation procedure and backward propagation procedure.
Please print out the size (dimension) of each layer (C1, P1, C2, P2, F3, F4, F5)
Please print your CNN architecture model as the above table.
Please optimize your model using a learning rate and number of iteration.
Please print out cost with number of iteration. It may take long time to calculate. You may limit the number of iteration
less than 10.

In [26]:
print X_train.shape
print neural_dict['Conv1']['W']
print neural_dict['Conv1']['B']
print neural_dict['Conv2']['W']
print neural_dict['Conv2']['B']

(1020, 64, 64, 3)
[[[[  4.17022005e-03   7.20324493e-03   1.14374817e-06   3.02332573e-03
      1.46755891e-03   9.23385948e-04   1.86260211e-03   3.45560727e-03]
   [  3.96767474e-03   5.38816734e-03   4.19194514e-03   6.85219500e-03
      2.04452250e-03   8.78117436e-03   2.73875932e-04   6.70467510e-03]
   [  4.17304802e-03   5.58689828e-03   1.40386939e-03   1.98101489e-03
      8.00744569e-03   9.68261576e-03   3.13424178e-03   6.92322616e-03]]

  [[  8.76389152e-03   8.94606664e-03   8.50442114e-04   3.90547832e-04
      1.69830420e-03   8.78142503e-03   9.83468338e-04   4.21107625e-03]
   [  9.57889530e-03   5.33165285e-03   6.91877114e-03   3.15515631e-03
      6.86500928e-03   8.34625672e-03   1.82882773e-04   7.50144315e-03]
   [  9.88861089e-03   7.48165654e-03   2.80443992e-03   7.89279328e-03
      1.03226007e-03   4.47893526e-03   9.08595503e-03   2.93614148e-03]]

  [[  2.87775339e-03   1.30028572e-03   1.93669579e-04   6.78835533e-03
      2.11628116e-03   2.65546659e-0

In [31]:
file_obj = h5py.File("Weights_A5_Tanmay_Bhatt.hdf5","w")

In [32]:
dataset = file_obj.create_dataset("C1W", data=neural_dict['Conv1']['W'])
dataset = file_obj.create_dataset("C1B", data=neural_dict['Conv1']['B'])
dataset = file_obj.create_dataset("C2W", data=neural_dict['Conv2']['W'])
dataset = file_obj.create_dataset("C2B", data=neural_dict['Conv2']['B'])
dataset = file_obj.create_dataset("FC1W", data=neural_dict['Fc1']['W'])
dataset = file_obj.create_dataset("FC1B", data=neural_dict['Fc1']['B'])
dataset = file_obj.create_dataset("FC2W", data=neural_dict['Fc2']['W'])
dataset = file_obj.create_dataset("FC2B", data=neural_dict['Fc2']['B'])
file_obj.close()

In [33]:
def test_foward_pass(Y_train,C1W,C1B,C2W,C2B,FC1W,FC1B,FC2W,FC2B):
    
    
    def test_calculate_loss(y,a):

        return (np.multiply(y,np.log(a)) + np.multiply((1-y),np.log(1-a)))
    
    def test_calculate_cost(y,a):
        m = X_train.shape[0]
        cost = -np.sum(test_calculate_loss(y,a))
        return cost/m
    
    
    def test_one_hot_encoding(mat):
        list_of_list = []
        for i in range(0,len(mat)):
            small_list = np.zeros(np.max(mat)+1)
            small_list[mat[i]] = 1
            list_of_list.append(small_list)
        result = np.array(list_of_list)
        return result

    neural_dict['Conv1']['W'] = C1W
    neural_dict['Conv1']['B'] = C1B
    neural_dict['Conv2']['W'] = C2W
    neural_dict['Conv2']['B'] = C2B
    neural_dict['Fc1']['W'] = FC1W
    neural_dict['Fc1']['B'] = FC1B
    neural_dict['Fc2']['W'] = FC2W
    neural_dict['Fc2']['B'] = FC2B
    
    
    neural_dict['Conv1']['params'] = {
            'padding' : 1,
            'stride' : 2
    }
    neural_dict['Pool1']['params'] = {
            'f' : 5,
            'stride' : 1
    }
    neural_dict['Conv2']['params'] = {
            'padding' : 0,
            'stride' : 2
    }
    neural_dict['Pool2']['params'] = {
            'f' : 5,
            'stride' : 1
    }
    
    ac1,c1 = conv_forward(X_train, neural_dict['Conv1']['W'], neural_dict['Conv1']['B'], neural_dict['Conv1']['params'],"ReLU")
    P1,cp1 = pool_forward(ac1, neural_dict['Pool1']['params'], mode = "max")
    ac2,c2 = conv_forward(P1, neural_dict['Conv2']['W'], neural_dict['Conv2']['B'], neural_dict['Conv2']['params'],"ReLU")
    P2,cp2 = pool_forward(ac2,  neural_dict['Pool2']['params'], mode = "average")
    
    f1 = []
    for item in P2:
        f1.append(item.flatten())
    f1 = np.asarray(f1)  
    
    W1 = neural_dict['Fc1']['W']
    W2 = neural_dict['Fc2']['W']
    B1 = neural_dict['Fc1']['B']
    B2 = neural_dict['Fc2']['B']
    
    z1 = np.dot(f1,W1.T).T + B1
    a1 = ReLU(z1)
    z2 = (np.dot(W2, a1)+ B2)
    a2 = sigmoid(z2).T
    
    Y_train_onehot = test_one_hot_encoding(Y_train)
    
    print "Training data shape : ", X_train.shape
    print "Convolution 1 shape : ", ac1.shape
    print "Pooling 1 shape : ",P1.shape
    print "Convolution 2 shape : ", ac2.shape
    print "Pooling 2 shape : ",P2.shape
    print "Flatten 1 shape : ",f1.shape
    print "Fully Connected 1 shape : ",a1.shape
    print "Fully Connected 2 shape : ",a2.shape
    
    print neural_dict['Conv1']['params']
    print neural_dict['Conv2']['params']
    print neural_dict['Pool1']['params']
    print neural_dict['Pool2']['params']
    print "Cost is :" , test_calculate_cost(Y_train_onehot,a2)

In [34]:
file_obj = h5py.File("Weights_A5_Tanmay_Bhatt.hdf5","r")

In [35]:
C1W = np.array(file_obj['C1W'])
C1B = np.array(file_obj['C1B'])
C2W = np.array(file_obj['C2W'])
C2B = np.array(file_obj['C2B'])
FC1W = np.array(file_obj['FC1W'])
FC1B = np.array(file_obj['FC1B'])
FC2W = np.array(file_obj['FC2W'])
FC2B = np.array(file_obj['FC2B'])
test_foward_pass(Y_train,C1W,C1B,C2W,C2B,FC1W,FC1B,FC2W,FC2B)

Training data shape :  (1020, 64, 64, 3)
Convolution 1 shape :  (1020, 32, 32, 8)
Pooling 1 shape :  (1020, 28, 28, 8)
Convolution 2 shape :  (1020, 13, 13, 16)
Pooling 2 shape :  (1020, 9, 9, 16)
Flatten 1 shape :  (1020, 1296)
Fully Connected 1 shape :  (108, 1020)
Fully Connected 2 shape :  (1020, 6)
{'padding': 1, 'stride': 2}
{'padding': 0, 'stride': 2}
{'stride': 1, 'f': 5}
{'stride': 1, 'f': 5}
Cost is : 2.73889074499
